<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
استخرمون نشه؟</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در بسیاری از وب‌سایت‌های گردشگری و رزرو اقامتگاه‌ها، کاربران می‌توانند تصاویر مختلفی از اقامتگاه‌های مختلف را مشاهده کنند. در این مسئله قصد داریم با استفاده از مجموعه‌ای از تصاویر اقامتگاه‌ها، مدلی طراحی کنیم که بتواند بر اساس تحلیل تصاویر یک اقامتگاه، وجود یا عدم وجود استخر در آن اقامتگاه را پیش‌بینی کند.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد می‌کنیم.
</font>
</p>

In [1]:
import pandas as pd
import numpy as np 

from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math
import spacy
from googletrans import Translator
from tqdm import tqdm

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import torch

import re



2024-10-20 01:15:50.467331: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-20 01:15:50.728024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 01:15:50.728076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 01:15:50.761081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 01:15:50.840397: I tensorflow/core/platform/cpu_feature_guar

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
مجموعه داده ارائه‌شده به صورت JSON حاوی اطلاعاتی درباره ۱۵۹۹ اقامتگاه است که شامل ویژگی‌های مختلفی می‌باشند. این مجموعه شامل یک لیست با کلید "rooms" است که اقامتگاه‌های موجود در آن قرار دارند. هر رکورد در این مجموعه داده نمایانگر یک اقامتگاه با خصوصیات و امکانات مربوط به آن است. در جدول زیر، کلیدها و توضیحات آن‌ها آورده شده‌اند.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

|کلید|توضیحات|
|:------:|:---:|
|id|شناسه یکتای اقامتگاه|
|title|عنوان اقامتگاه|
|description|توضیحاتی در مورد اقامتگاه|
|province|اطلاعات استان شامل شناسه و نام استان|
|city|اطلاعات شهر شامل شناسه و نام شهر|

</font>
</div>
</center>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
نمونه‌ای از این داده‌ها به صورت زیر است:
</font>
</p>

```json
{
  "rooms": [
    {
      "id": 3202100,
      "title": "اجاره سوئیت در جاده جواهرده - اشکونکوه",
      "description": "این سوئیت بدون اتاق خواب با بالکنی دلباز و چشم اندازی زیبا در طبقه همکف یک ساختمان دو طبقه واقع شده است. فاصله اقامتگاه تا سوپرمارکت و نانوایی به ترتیب حدود ۵۰ و ۵۰۰ متر می باشد.",
      "province": {
        "id": "p26",
        "name": "مازندران"
      },
      "city": {
        "id": 303,
        "name": "رامسر"
      },
    },
    ...
  ]
}
```


<h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size="3">
    ابتدا نیاز است فایل‌های مجموعه‌‌ی داده را بخوانید. ویژگی‌های آموزشی در فایل <code>train.json</code> و دایرکتوری مربوط به تصاویر نمونه‌های آزمون که باید دسته‌ی آن‌ها را پیش‌بینی کنید، در فایل <code>test.json</code> ذخیره شده‌اند. در صورت نیاز می‌توانید بخشی از داده‌های آموزشی را به عنوان داده‌های اعتبارسنجی جدا کنید. توجه داشته باشید که ستون‌های <code>description</code> و <code>title</code> از داده‌های آزمون حذف شده‌اند و هدف اصلی این فایل، دسترسی به دایرکتوری تصاویر داده‌های آزمون است. دایرکتوری هر تصویر برابر با <code>id</code> هر اقامتگاه است و داخل پوشه <code>pictures</code> قرار دارد.
</font>
</p>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <font face="vazir">
        <span style="color:orange"><b>نکته:</b></span>
        حجم فایل‌های این سوال حدود ۳.۸ گیگابایت است. در صورتی‌که بارگذاری این فایل‌ها در محیط کولب برای‌تان دشوار است می‌توانید با استفاده از دستور زیر این دو فایل را به‌صورت مستقیم در محیط کولب بارگیری کنید.
        همچنین می‌توانید از لینک درایو زیر فایل‌ها را دانلود کرده و بر روی محیط دستگاه خود از آن استفاده کنید.
    </font>
</p>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <font face="vazir">
        <a href="https://drive.google.com/file/d/1b8O_a6ywcsbLqJAGDGCkePrdn1cFlXl0/view?usp=sharing" target="_blank" style="color:blue;">لینک دانلود از گوگل درایو</a>
    </font>
</p>





In [2]:
train_data = pd.read_json('data_pool/train.json')
print(train_data)

                                                  rooms
0     {'id': 3175858, 'title': 'رزرو ویلا با استخر ر...
1     {'id': 3237321, 'title': 'اجاره ویلا استخردار ...
2     {'id': 3154228, 'title': 'اجاره ویلا با استخر ...
3     {'id': 3169850, 'title': 'رزرو سوئیت در جواهرد...
4     {'id': 3207557, 'title': 'اجاره سوئیت در جاده ...
...                                                 ...
1594  {'id': 3167459, 'title': 'اجاره ویلا استخردار ...
1595  {'id': 3207406, 'title': 'اجاره ویلا استخردار ...
1596  {'id': 3172620, 'title': 'اجاره ویلا چابکسر - ...
1597  {'id': 3237948, 'title': 'اجاره سوئیت در جواهر...
1598  {'id': 3229812, 'title': 'اجاره خانه با ویو کو...

[1599 rows x 1 columns]


In [3]:
test_data = pd.read_json('data_pool/test.json')
print(test_data)

               rooms
0    {'id': 3160664}
1    {'id': 3195184}
2    {'id': 3224078}
3    {'id': 3233712}
4    {'id': 3201449}
..               ...
856  {'id': 3232618}
857  {'id': 3230930}
858  {'id': 3171083}
859  {'id': 3168538}
860  {'id': 3168123}

[861 rows x 1 columns]


<h2 align="right" style="line-height: 200%; font-family: Vazir; color: #0099cc;">
    پیش‌پردازش و مهندسی ویژگی
</h2>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height: 200%; font-family: Vazir; font-size: medium;">
    در این بخش، باید با استفاده از توصیفات و تیتر هر اقامت‌گاه، وجود یا عدم وجود استخر را استخراج کنید. می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>ما در این بخش از کتابخانه re معروف استفاده کردیم
</p>


In [4]:

def detect_pool(text):
    pool_keywords = ["استخر","سونا","جکوزی","استخردار","استخر‌"]
    for keyword in pool_keywords:
        if re.search(keyword, text):
            return 1  
    return 0  

results = []
texts=[]
for i in range (int(train_data.shape[0])):
    text = train_data["rooms"][i]['title'] + train_data["rooms"][i]['description']
    train_data.loc[i, 'id'] = int(train_data["rooms"][i]['id'])
    texts.append(text)
  
for text in tqdm(texts, desc="Processing texts"):
    
    result = detect_pool(text)
    results.append(result)

train_data["pool"]=results
train_data = train_data[["pool", "id"]]
train_data.head()

Processing texts: 100%|██████████| 1599/1599 [00:00<00:00, 334048.52it/s]


,pool,id
0,1,3175858.0
1,1,3237321.0
2,1,3154228.0
3,0,3169850.0
4,0,3207557.0


<p dir="rtl" style="direction: rtl; text-align: justify; line-height: 200%; font-family: Vazir; font-size: medium;">
    اینجا کار خاصی انجام نشده فقط نکته ای که هست اینه که بجای اینکه از labelEncoder استفاده کنیم که آیدی هارو داشته باشیم ، متاسفانه به دلیل فرصت کم مجبود شدیم تایپ داده رو عوض کنیم که در مراحل بعد تبدیل به None  بشه
    <br>
</p>


In [5]:
train_data["id"]=train_data["id"].astype(str)
train_data.head()

,pool,id
0,1,3175858.0
1,1,3237321.0
2,1,3154228.0
3,0,3169850.0
4,0,3207557.0


<p dir="rtl" style="direction: rtl; text-align: justify; line-height: 200%; font-family: Vazir; font-size: medium;">
تصاویر مربوط به هر اقامتگاه رو میچسبونیم به هم در یه تصویر، علت اش واضحه نه؟ یه کتاب رو نمیشه از رو جلدش قضاوت کرد.
    <br>
</p>

In [6]:
i


def create_image_dataframe(data, image_dir, output_size=(256, 256), cell_size=(64, 64)):
    image_paths = []
    labels = []

    folder_paths = {str(int(float(row['id']))): row['pool'] for _, row in data.iterrows()}

    def merge_images_in_folder(folder_path, output_size, cell_size):
        """
        Merge all images in the folder into a single image with dynamic grid layout.
        """
        image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('jpg')]
        
        if len(image_files) == 0:
            print(f"No images found in {folder_path}")
            return None

        num_images = len(image_files)
        grid_cols = math.ceil(math.sqrt(num_images)) 
        grid_rows = math.ceil(num_images / grid_cols) 

        images = [Image.open(img).resize(cell_size) for img in image_files]

        merged_image = Image.new('RGB', (grid_cols * cell_size[0], grid_rows * cell_size[1]))

        for idx, img in enumerate(images):
            row = idx // grid_cols
            col = idx % grid_cols
            merged_image.paste(img, (col * cell_size[0], row * cell_size[1]))

        merged_image = merged_image.resize(output_size)

        return merged_image
    
    for id_folder, label in folder_paths.items():
        folder_path = os.path.join(image_dir, id_folder)
        
        if os.path.exists(folder_path): 
            merged_image = merge_images_in_folder(folder_path, output_size, cell_size)
            if merged_image:
                merged_image_path = os.path.join(image_dir+"/images/", f"{id_folder}.jpg")
                merged_image.save(merged_image_path)  
                image_paths.append(merged_image_path)  
                labels.append(label)  
        else:
            print(f"Warning: Folder {folder_path} not found.")
    
    return pd.DataFrame({'image_path': image_paths, 'pool': labels})

# df = create_image_dataframe(data, 'data_pool/train', output_size=(512, 512), cell_size=(128, 128))

train_data['id'] = train_data['id'].astype(str)
train_data['pool'] = train_data['pool'].astype(int)

train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

image_dir = 'data_pool/train'

train_df = create_image_dataframe(train_df, image_dir)
val_df = create_image_dataframe(val_df, image_dir)


 <p dir="rtl" style="direction: rtl; text-align: justify; line-height: 200%; font-family: Vazir; font-size: medium;">
دو تا خط پایین کار خاصی نکردم :) برای اینکه نخام دوباره پیش پردازش داده ها رو داخل ران هام انجام بدم سیو کردم دیتاست هامو   
    <br>
</p>

In [7]:
train_df.to_csv("train_df.csv")
val_df.to_csv("val_df.csv")


In [8]:
import pandas as pd
val_df=pd.read_csv("val_df.csv")
train_df=pd.read_csv("train_df.csv")


 <p dir="rtl" style="direction: rtl; text-align: justify; line-height: 200%; font-family: Vazir; font-size: medium;">
پیش پردازش نهایی واسه ارسال داده ها به مدل    <br>
</p>

In [9]:
import tensorflow as tf
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)  
    image = tf.image.decode_jpeg(image_string, channels=3)  
    image = tf.image.resize(image, [256, 256])  
    image = image / 255.0  
    return image, label

total_size = len(train_df)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

print(list(train_df["image_path"]))

dataset = tf.data.Dataset.from_tensor_slices((list(train_df["image_path"]), list(train_df["pool"])))

dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.shuffle(buffer_size=total_size, seed=42)



train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)


train_dataset = train_dataset.batch(1)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.batch(1)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


print(f"Total size: {total_size}")
print(f"Training dataset size: {train_size}, Validation dataset size: {val_size}")
print(f"Number of batches in training dataset: {len(train_dataset)}")
print(f"Number of batches in validation dataset: {len(val_dataset)}")


['data_pool/train/images/3216606.jpg', 'data_pool/train/images/3223930.jpg', 'data_pool/train/images/3168185.jpg', 'data_pool/train/images/3187761.jpg', 'data_pool/train/images/3229610.jpg', 'data_pool/train/images/3228960.jpg', 'data_pool/train/images/3197932.jpg', 'data_pool/train/images/3206982.jpg', 'data_pool/train/images/3220538.jpg', 'data_pool/train/images/3193462.jpg', 'data_pool/train/images/3183382.jpg', 'data_pool/train/images/3239220.jpg', 'data_pool/train/images/3146921.jpg', 'data_pool/train/images/3208556.jpg', 'data_pool/train/images/3206305.jpg', 'data_pool/train/images/3192439.jpg', 'data_pool/train/images/3176181.jpg', 'data_pool/train/images/3229066.jpg', 'data_pool/train/images/3211064.jpg', 'data_pool/train/images/3198514.jpg', 'data_pool/train/images/3171145.jpg', 'data_pool/train/images/3213655.jpg', 'data_pool/train/images/3210633.jpg', 'data_pool/train/images/3217238.jpg', 'data_pool/train/images/3212232.jpg', 'data_pool/train/images/3235368.jpg', 'data_pool/

2024-10-20 01:22:00.135212: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-20 01:22:00.139427: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-20 01:22:00.146258: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Total size: 1279
Training dataset size: 1023, Validation dataset size: 256
Number of batches in training dataset: 1023
Number of batches in validation dataset: 256


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    حال که داده‌ها را پیش‌پردازش کرده‌ایم، و دسته‌بندی وجود یا عدم وجود استخر را از متن انجام داده‌ایم، وقت آن است که مدلی آموزش دهیم که بتواند متغیر هدف این مسئله یعنی وجود یا عدم وجود استخر را پیش‌بینی کند.
    <br>
</font>
</p>

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras import layers, models

attributes = ['pool']



def create_model():
    inputs = layers.Input(shape=(256, 256, 3))
    x = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, (9,9), activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(1, activation='softmax')(x)  
    model = models.Model(inputs, outputs)
    return model



In [11]:

model = create_model()
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath='best_model.h5',  
    monitor='val_loss',        
    save_best_only=True,      
    save_weights_only=False,  
    mode='min',               
    verbose=1   )             

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=7,
    callbacks=[checkpoint_callback]  
)

Epoch 1/7


2024-10-20 01:22:01.737651: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-10-20 01:22:01.866492: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-20 01:22:02.318499: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe2a03d58d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-20 01:22:02.318524: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-10-20 01:22:02.318528: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-10-20 01:22:02.326440: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1729374722.379163   19746 device_comp

1023/1023 [==============================] - ETA: 0s - loss: 1.1113 - accuracy: 0.5455
Epoch 1: val_loss improved from inf to 0.64976, saving model to best_model.h5


/home/absi1377/anaconda3/envs/cornac/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1023/1023 [==============================] - 29s 26ms/step - loss: 1.1113 - accuracy: 0.5455 - val_loss: 0.6498 - val_accuracy: 0.5469
Epoch 2/7
1022/1023 [============================>.] - ETA: 0s - loss: 0.7594 - accuracy: 0.5499
Epoch 2: val_loss improved from 0.64976 to 0.63502, saving model to best_model.h5
1023/1023 [==============================] - 28s 27ms/step - loss: 0.7595 - accuracy: 0.5494 - val_loss: 0.6350 - val_accuracy: 0.5664
Epoch 3/7
1023/1023 [==============================] - ETA: 0s - loss: 0.6835 - accuracy: 0.5347
Epoch 3: val_loss improved from 0.63502 to 0.60552, saving model to best_model.h5
1023/1023 [==============================] - 29s 28ms/step - loss: 0.6835 - accuracy: 0.5347 - val_loss: 0.6055 - val_accuracy: 0.5586
Epoch 4/7
1021/1023 [============================>.] - ETA: 0s - loss: 0.5764 - accuracy: 0.5446
Epoch 4: val_loss improved from 0.60552 to 0.49201, saving model to best_model.h5
1023/1023 [==============================] - 26s 25ms/step

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>f1_score</code> نام دارد.
    <br>
</font>
</p>

In [16]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import numpy as np

model.load_weights('best_model.h5')

val_predictions = model.predict(val_dataset)

val_predictions = (val_predictions > 0.5).astype(int)

# Step 4: Get the true labels from the validation dataset
val_true_labels = np.concatenate([y for x, y in val_dataset], axis=0)

# Step 5: Calculate the F1 score
f1 = f1_score(val_true_labels, val_predictions, average='binary')

print(f'F1 Score: {f1}')


256/256 [==============================] - 1s 3ms/step
F1 Score: 0.7295285359801489
